In [6]:
import os

In [7]:
%pwd

'/Users/suyash/Desktop/Intent-classification-/research'

In [8]:
os.chdir('../')

In [9]:
%pwd

'/Users/suyash/Desktop/Intent-classification-'

In [10]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path


In [11]:
from src.ic.constants import *
from src.ic.utils.common import read_yaml,create_directories

In [12]:
class ConfigurationManager:
    def __init__(self,
                 config_path=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH):
        self.config=read_yaml(config_path)
        self.paramss=read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self)-> DataTransformationConfig:
        config=self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config=DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            tokenizer_name=config.tokenizer_name
        )

        return data_transformation_config



In [14]:
import pandas as pd

In [16]:
class DataTransformation:
    def __init__(self,config:DataTransformationConfig):
        self.config=config
        self.tokenizer=De.from_pretrained(config.tokenizer_name)

    def convert_examples_to_features(self,example_batch):
        input_encodings = self.tokenizer(example_batch['intents'] )

        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(example_batch['utterneces'])

        return {
            'input_ids' : input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']
        }
    
    def convert(self,dataset):
        df=pd.read_csv('/Users/suyash/Desktop/Intent-classification-/artifacts/data_ingestion/train/train.csv')
        



    

In [ ]:
config=ConfigurationManager()
data_transformation_config=config.get_data_transformation_config()
data_transformation=DataTransformation(config=data_transformation_config)
data_transformation.convert()
